In [1]:
# Notebook to preprocess the datasets and create new datasets required for the analysis.
# import required packages
import numpy as np
import pandas as pd

drivers=pd.read_csv('.//data//drivers.csv')
champresults=pd.read_csv('.//data//driver_standings.csv')
qualiresults=pd.read_csv('.//data//qualifying.csv')
racedata=pd.read_csv('.//data//races.csv')
raceresults=pd.read_csv('.//data//results.csv')

Build array with the raceId of the first and last race each year.

In [2]:
racedataPerYear=racedata.groupby('year')
listracesperyear=[racedataPerYear.get_group(x) for x in racedataPerYear.groups]
raceIdyearSplitter=np.array([[x['year'].min(), x['raceId'].min(), x['raceId'].max()] for x in listracesperyear ])
#Convert the previous list to a dictionary for easier access in later steps
raceIdyearDictio={'{}'.format(x[0]):x[1:] for i, x in enumerate(raceIdyearSplitter)}

Select a year from which we will start considering the results and then build an array with each driver in each year. If a driver competed several years, they will appear several times since each competed season will be a different data point.

In [3]:
minYearToConsider=1990
i, = np.where( raceIdyearSplitter[:,0] == 1990)
data_array=[]
for j in range(int(i),len(raceIdyearSplitter[:,0])-2,1): #I exclude the year 2023 due to being incomplete.
    for ind, row in champresults[champresults['raceId']==raceIdyearSplitter[j,2]][['driverId','position']].iterrows():
        data_array.append([raceIdyearSplitter[j,0], row['driverId'],row['position']])

Convert the array to a Pandas DataFrame. This is the basic Data Frame we will use.
Now, we will append columns describing the season of each driver in each year: race results, quali results.

In [4]:
driverData=pd.DataFrame(data_array,columns=['Year','Driver','Championship Position'])

In [5]:
driverData.head()

,Year,Driver,Championship Position
0,1990,65,35
1,1990,152,39
2,1990,88,12
3,1990,95,5
4,1990,77,4


Generate the column with the race results for each driver in each year.
Consider the finishing position without considering whether they completed the race distance or not. This could generate problems with outliers where they finish high enough to score points but didn't complete enough laps.
Those cases should be rare enough to have negligible impact.

In [6]:
driverData=driverData.assign(RaceResults=None)
for i, dri in driverData.iterrows():
    resdri=raceresults[(raceresults['raceId']>=raceIdyearDictio[str(dri['Year'])][0])*(raceresults['raceId']<=raceIdyearDictio[str(dri['Year'])][1])*(raceresults['driverId']==dri['Driver'])]['positionOrder'].reset_index(drop=True).to_numpy()
    driverData.at[i,'RaceResults']=resdri.astype(float)

Similar, but for grid starting position instead of final race position

In [7]:
driverData=driverData.assign(GridPos=None)
for i, dri in driverData.iterrows():
    qualidri=raceresults[(raceresults['raceId']>=raceIdyearDictio[str(dri['Year'])][0])*(raceresults['raceId']<=raceIdyearDictio[str(dri['Year'])][1])*(raceresults['driverId']==dri['Driver'])]['grid'].reset_index(drop=True).to_numpy()
    driverData.at[i,'GridPos']=qualidri.astype(float)

And for qualifying results. The original database is not so complete regarding qualifying results.

In [8]:
driverData=driverData.assign(QualiRes=None)
for i, dri in driverData.iterrows():
    quaResdri=qualiresults[(qualiresults['raceId']>=raceIdyearDictio[str(dri['Year'])][0])*(qualiresults['raceId']<=raceIdyearDictio[str(dri['Year'])][1])*(qualiresults['driverId']==dri['Driver'])]['position'].reset_index(drop=True).to_numpy()
    driverData.at[i,'QualiRes']=quaResdri.astype(float)
driverData.head(20)

,Year,Driver,Championship Position,RaceResults,GridPos,QualiRes
0,1990,65,35,"[12.0, 17.0]","[12.0, 18.0]",[]
1,1990,152,39,"[33.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[]
2,1990,88,12,"[16.0, 21.0, 21.0, 21.0, 12.0, 23.0, 7.0, 6.0,...","[18.0, 18.0, 15.0, 15.0, 18.0, 19.0, 14.0, 9.0...",[]
3,1990,95,5,"[17.0, 4.0, 15.0, 9.0, 3.0, 2.0, 18.0, 15.0, 1...","[17.0, 5.0, 5.0, 7.0, 7.0, 4.0, 1.0, 1.0, 4.0,...",[]
4,1990,77,4,"[18.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, 14.0, 3.0...","[1.0, 2.0, 2.0, 5.0, 2.0, 1.0, 2.0, 3.0, 2.0, ...",[]
5,1990,148,22,"[19.0, 23.0, 16.0, 7.0, 15.0, 15.0, 29.0, 30.0...","[23.0, 22.0, 23.0, 20.0, 21.0, 23.0, 0.0, 0.0,...",[]
6,1990,129,31,"[20.0, 25.0, 14.0, 28.0, 13.0, 19.0, 31.0, 27....","[8.0, 21.0, 22.0, 0.0, 15.0, 20.0, 0.0, 0.0, 0...",[]
7,1990,110,27,"[21.0, 26.0, 18.0, 14.0, 16.0, 13.0, 19.0, 24....","[3.0, 9.0, 17.0, 12.0, 25.0, 15.0, 21.0, 23.0,...",[]
8,1990,101,34,"[30.0, 18.0, 30.0, 25.0, 15.0, 28.0, 21.0, 28....","[0.0, 25.0, 0.0, 21.0, 25.0, 0.0, 21.0, 0.0, 2...",[]
9,1990,139,26,"[23.0, 25.0, 24.0, 26.0, 26.0, 11.0, 25.0, 10....","[21.0, 9.0, 19.0, 18.0, 24.0, 19.0, 23.0, 13.0...",[]


Add the number of races entered by each driver, the average final position, its standard deviation and the median final position. Also the average, standard deviation and median for the starting position. 

In [9]:
driverData['NumRaces']=driverData['RaceResults'].apply(len).values
driverData['AvgFinalPos']=driverData['RaceResults'].apply(np.mean).values
driverData['StdDevFinalPos']=driverData['RaceResults'].apply(np.std).values
driverData['MedianFinalPos']=driverData['RaceResults'].apply(np.median).values
driverData['AvgStartPos']=driverData['GridPos'].apply(np.mean).values
driverData['StdDevStartPos']=driverData['GridPos'].apply(np.std).values
driverData['MedianStartPos']=driverData['GridPos'].apply(np.median).values

In [10]:
driverData.head(5)

,Year,Driver,Championship Position,RaceResults,GridPos,QualiRes,NumRaces,AvgFinalPos,StdDevFinalPos,MedianFinalPos,AvgStartPos,StdDevStartPos,MedianStartPos
0,1990,65,35,"[12.0, 17.0]","[12.0, 18.0]",[],2,14.500000,2.500000,14.5,15.0000,3.000000,15.0
1,1990,152,39,"[33.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],12,34.166667,0.986013,35.0,0.0000,0.000000,0.0
2,1990,88,12,"[16.0, 21.0, 21.0, 21.0, 12.0, 23.0, 7.0, 6.0,...","[18.0, 18.0, 15.0, 15.0, 18.0, 19.0, 14.0, 9.0...",[],16,15.687500,7.042006,16.0,15.2500,4.100305,15.0
3,1990,95,5,"[17.0, 4.0, 15.0, 9.0, 3.0, 2.0, 18.0, 15.0, 1...","[17.0, 5.0, 5.0, 7.0, 7.0, 4.0, 1.0, 1.0, 4.0,...",[],16,10.187500,7.178255,11.5,4.6875,3.652375,4.0
4,1990,77,4,"[18.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, 14.0, 3.0...","[1.0, 2.0, 2.0, 5.0, 2.0, 1.0, 2.0, 3.0, 2.0, ...",[],16,7.437500,6.470788,4.0,2.6875,1.210307,2.0


In [11]:
driverData.tail(5)

,Year,Driver,Championship Position,RaceResults,GridPos,QualiRes,NumRaces,AvgFinalPos,StdDevFinalPos,MedianFinalPos,AvgStartPos,StdDevStartPos,MedianStartPos
842,2021,842,9,"[17.0, 7.0, 10.0, 10.0, 6.0, 3.0, 7.0, 20.0, 9...","[5.0, 5.0, 9.0, 12.0, 6.0, 4.0, 6.0, 6.0, 6.0,...","[5.0, 5.0, 9.0, 12.0, 6.0, 4.0, 6.0, 6.0, 6.0,...",22,9.363636,5.200922,7.0,6.363636,3.023325,6.0
843,2021,849,17,"[18.0, 20.0, 18.0, 16.0, 15.0, 16.0, 18.0, 17....","[17.0, 14.0, 18.0, 19.0, 18.0, 16.0, 16.0, 16....","[17.0, 14.0, 18.0, 19.0, 18.0, 16.0, 16.0, 16....",22,15.545455,3.172714,16.0,15.227273,3.918857,16.0
844,2021,4,10,"[19.0, 10.0, 8.0, 17.0, 13.0, 6.0, 8.0, 9.0, 1...","[9.0, 15.0, 13.0, 10.0, 17.0, 8.0, 9.0, 8.0, 1...","[9.0, 15.0, 13.0, 10.0, 17.0, 9.0, 9.0, 9.0, 1...",22,9.909091,4.316028,9.0,10.500000,3.738619,10.0
845,2021,853,21,"[20.0, 17.0, 19.0, 19.0, 17.0, 14.0, 20.0, 18....","[19.0, 19.0, 20.0, 20.0, 19.0, 18.0, 18.0, 20....","[20.0, 19.0, 20.0, 20.0, 19.0, 18.0, 18.0, 20....",22,17.863636,1.713462,18.0,18.500000,1.530003,19.0
846,2021,9,20,"[15.0, 14.0]","[16.0, 17.0]","[18.0, 19.0]",2,14.500000,0.500000,14.5,16.500000,0.500000,16.5


Export the cleaned data CSV for easy reuse

In [12]:
driverData.to_csv('./data/driver_clean_data.csv',index=False)